# <center>NYPD Motor Vehicle Collision Data</center><a id='Top'></a>
## <center>supported by a <a href="https://github.com/CorkCork/NYPD-Motor-Vehicle-Collisions/blob/master/README.md"> README file </a></center></center> 
### <center> Alan Leidner, Esin Alpturk, Kristin Medlin</center>
#### <center> November 10, 2019</center>

### Overview

The "Motor Vehicle Collisions - Crashes" dataset available through New York City's OpenData program contains information about reported vehicle crashes in NYC. Each row contains details on a single crash event. 

The data contains records from 2012 to today, with data updated on a daily basis. At the time of this writing, there are 1.59 million rows, each row representing a crash event, and over 29 columns which represent date, time, borough, zip code, latitude, etc...

In this notebook we will analyze this data as follows:

1. [Importing](#Importing)
2. [Understanding](#Understanding)
    - [Column Contents](#column_contents)
    - [Descriptive Statistics](#descriptive_statistics)
    - [Columns Missing Data](#empty)
3. [Transforming](#Transforming)
    - [Dropping Columns](#Drop)
    - [Renaming Data](#Renaming)
    - [Redundant Columns](#Redundant)
    - [Data Types](#data_type)
4. [Closing Thoughts](#conclusion)
5. [End of Document](#Bottom)

* The dataset can be found by following this link: https://data.cityofnewyork.us/Public-Safety/NYPD-Motor-Vehicle-Collisions-Crashes/h9gi-nx95

### Importing the Data<a id='Importing'></a>

Let's begin by importing a few libraries we will use later in the notebook, and then bring the first two million rows of NYPD Motor Vehicle Collision Data using pandas. We are over-estimating the number of rows, to leave room for more data if this is run in the future.

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import sys
from IPython.core.display import display, HTML

In [2]:
datanyc = pd.read_csv("https://data.cityofnewyork.us/resource/h9gi-nx95.csv?$limit=2000000")

/Users/leidner/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


And let's pull up the data dictionary supplied by the OpenData website for reference.

In [3]:
data_dict = pd.read_excel("https://data.cityofnewyork.us/api/views/h9gi-nx95/files/2e58023a-21a6-4c76-b9e8-0101bf7509ca?download=true&filename=MVCollisionsDataDictionary.xlsx",
                         sheet_name='Column Info')
data_dict.head()

,Data Dictionary - Column Information,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Table Name,Column Name,Column Description,Primary Key or Foreign Key,"Additional Notes \n(where applicable, includes..."
1,MV-Collisions - Crash,UNIQUE_ID,Unique record code generated by system,Primary Key for the crash table,NaN
2,MV-Collisions - Crash,ACCIDENT_DATE,Occurrence date of collision,NaN,NaN
3,MV-Collisions - Crash,ACCIDENT_TIME,Occurrence time of collision,NaN,NaN
4,MV-Collisions - Crash,BOROUGH,Borough where collision occurred,NaN,NaN


### Understanding the Data <a id='Understanding'></a>
Let's look the first few rows of the dataset. 

In [4]:
pd.set_option('display.max_columns', None) # This allows us to view all columns in a dataframe when called
pd.set_option('display.max_rows', 200) # This returns 200 rows at max to prevent accidents when writing code
datanyc.head()

,date,time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2013-10-05T00:00:00.000,17:00,QUEENS,11355,40.754769,-73.829960,POINT (-73.8299602 40.7547687),MAPLE AVENUE,FRAME PLACE,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,252455,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE,NaN,NaN,NaN
1,2013-10-27T00:00:00.000,8:00,QUEENS,11004,40.743511,-73.717405,POINT (-73.7174046 40.7435109),LITTLE NECK PARKWAY,80 AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,225948,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN
2,2013-10-15T00:00:00.000,10:30,MANHATTAN,10013,40.723121,-74.006234,POINT (-74.0062341 40.7231208),VARICK STREET,GRAND STREET,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,4085,PASSENGER VEHICLE,VAN,NaN,NaN,NaN
3,2013-10-29T00:00:00.000,15:00,BROOKLYN,11220,40.633235,-74.011907,POINT (-74.0119074 40.6332345),64 STREET,8 AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Fatigued/Drowsy,Unspecified,NaN,NaN,NaN,143798,SPORT UTILITY / STATION WAGON,UNKNOWN,NaN,NaN,NaN
4,2013-10-06T00:00:00.000,5:52,NaN,NaN,40.853248,-73.835281,POINT (-73.8352807 40.8532485),NaN,NaN,NaN,1.0,0.0,0,0,0,0,1,0,Unspecified,NaN,NaN,NaN,NaN,2913656,PASSENGER VEHICLE,NaN,NaN,NaN,NaN


... And get some overall information about the contents of the data. <a id='column_contents'></a>

In [5]:
pd.options.display.max_info_rows = 5000000
datanyc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600038 entries, 0 to 1600037
Data columns (total 29 columns):
date                             1600038 non-null object
time                             1600038 non-null object
borough                          1115106 non-null object
zip_code                         1114912 non-null object
latitude                         1404135 non-null float64
longitude                        1404135 non-null float64
location                         1404135 non-null object
on_street_name                   1286755 non-null object
off_street_name                  1065533 non-null object
cross_street_name                221118 non-null object
number_of_persons_injured        1600021 non-null float64
number_of_persons_killed         1600007 non-null float64
number_of_pedestrians_injured    1600038 non-null int64
number_of_pedestrians_killed     1600038 non-null int64
number_of_cyclist_injured        1600038 non-null int64
number_of_cyclist_killed        

Each column should in theory contain 159,9576 values. There are some columns with considerably fewer entries. Let's find the percentage of the missing values and see which columns have the most amount of missing values. We will get a mean of missing values and then round it to 4 decimal place.

In [6]:
pd.set_option('display.max_columns', 29)
datanyc.isnull().mean().round(4) * 100

date                              0.00
time                              0.00
borough                          30.31
zip_code                         30.32
latitude                         12.24
longitude                        12.24
location                         12.24
on_street_name                   19.58
off_street_name                  33.41
cross_street_name                86.18
number_of_persons_injured         0.00
number_of_persons_killed          0.00
number_of_pedestrians_injured     0.00
number_of_pedestrians_killed      0.00
number_of_cyclist_injured         0.00
number_of_cyclist_killed          0.00
number_of_motorist_injured        0.00
number_of_motorist_killed         0.00
contributing_factor_vehicle_1     0.26
contributing_factor_vehicle_2    13.43
contributing_factor_vehicle_3    93.53
contributing_factor_vehicle_4    98.65
contributing_factor_vehicle_5    99.66
collision_id                      0.00
vehicle_type_code1                0.33
vehicle_type_code2       

Wow! Some columns have a lot of missing values. 

For some it makes sense. `contributing_factor_vehicle_2` or `contributing_factor_vehicle_3` may be missing simply because there were no second or third vehicle or factors contributing to the collision.  Other than those, for example `borough` column has 30% of its values missing. 

It looks like `contributing_factor_vehicle_3`, `contributing_factor_vehicle_4`, `contributing_factor_vehicle_5` and `vehicle_type_code_3`, `vehicle_type_code_4`, `vehicle_type_code_5` have very few values compared to the others. We will take a closer look at them when we start transforming our data.

For now, we will use the `describe` function to generate some descriptive statistics. This will work on numeric and object series, and may point out any glaring wholes in the data. <a id='descriptive_statistics'></a>

In [7]:
datanyc.describe()

,latitude,longitude,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,collision_id
count,1.404135e+06,1.404135e+06,1.600021e+06,1.600007e+06,1.600038e+06,1.600038e+06,1.600038e+06,1.600038e+06,1.600038e+06,1.600038e+06,1.600038e+06
mean,4.069426e+01,-7.387677e+01,2.625728e-01,1.167495e-03,5.050443e-02,6.312350e-04,2.082138e-02,8.437300e-05,1.913917e-01,4.543642e-04,2.769031e+06
std,1.101651e+00,2.287245e+00,6.600770e-01,3.612315e-02,2.315786e-01,2.570674e-02,1.439587e-01,9.252886e-03,6.224473e-01,2.319280e-02,1.505629e+06
min,0.000000e+00,-2.013600e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.200000e+01
25%,4.066880e+01,-7.397722e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.027459e+06
50%,4.072258e+01,-7.392986e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.436142e+06
75%,4.076796e+01,-7.386702e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.836383e+06
max,4.334444e+01,0.000000e+00,4.300000e+01,8.000000e+00,2.700000e+01,6.000000e+00,4.000000e+00,2.000000e+00,4.300000e+01,5.000000e+00,4.237285e+06


Obviously averages and standard deviations don't tell a lot about latitude and longitude, but why we don't have full data counts in those columns? The rest of analysis doesn't appear to have obvious problems.

Let's take a look at some of the empty `latitude` columns using the `isnull` function. <a id='empty'></a>

In [8]:
datanyc[datanyc['latitude'].isnull()].head(20)

,date,time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
5,2013-10-29T00:00:00.000,22:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0,0,0,0,1,0,Unspecified,Unspecified,NaN,NaN,NaN,3090413,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
6,2013-10-30T00:00:00.000,14:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,2913734,UNKNOWN,VAN,NaN,NaN,NaN
16,2013-10-26T00:00:00.000,0:25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Fatigued/Drowsy,Unspecified,NaN,NaN,NaN,2927683,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
26,2013-10-29T00:00:00.000,9:20,NaN,NaN,NaN,NaN,NaN,20 AVENUE,WHITESTONE EXPRESSWAY,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,252788,OTHER,OTHER,NaN,NaN,NaN
31,2013-10-13T00:00:00.000,22:16,NaN,NaN,NaN,NaN,NaN,WEST 142 STREET,RIVERSIDE DRIVE,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,65997,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN
32,2013-10-15T00:00:00.000,8:30,NaN,NaN,NaN,NaN,NaN,GRAND CENTRAL PARKWAY E SVC RD,JEWEL AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,Unspecified,NaN,NaN,267038,SPORT UTILITY / STATION WAGON,SPORT UTILITY / STATION WAGON,LIVERY VEHICLE,NaN,NaN
40,2013-10-03T00:00:00.000,7:45,NaN,NaN,NaN,NaN,NaN,NORTH GANNON AVENUE,VICTORY BOULEVARD,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,291579,PASSENGER VEHICLE,UNKNOWN,NaN,NaN,NaN
41,2013-10-28T00:00:00.000,8:05,NaN,NaN,NaN,NaN,NaN,SEAGIRT BOULEVARD,BEACH 9 STREET,NaN,1.0,0.0,1,0,0,0,0,0,Driver Inattention/Distraction,NaN,NaN,NaN,NaN,203345,PASSENGER VEHICLE,NaN,NaN,NaN,NaN
44,2013-10-23T00:00:00.000,13:20,NaN,NaN,NaN,NaN,NaN,CROSS ISLAND PARKWAY SR SOUTH,CLEARVIEW EXPRESSWAY,NaN,2.0,0.0,0,0,0,0,2,0,Failure to Yield Right-of-Way,Unspecified,NaN,NaN,NaN,252698,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
49,2013-10-29T00:00:00.000,19:05,NaN,NaN,NaN,NaN,NaN,EAST 120 STREET,2 AVENUE,NaN,1.0,0.0,0,0,1,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,61608,OTHER,BICYCLE,NaN,NaN,NaN


Seems like those rows have valid data. We wouldn't delete these rows, as the injury/fatality data may be useful, but we would drop them for location purposes.

If we *really* had some time, we would write/find a program to reverse map the on_street_name to the other fields.

For now, we'll pivot to take a closer look at vehicle types.

In [9]:
datanyc['vehicle_type_code1'].value_counts().head(20)

PASSENGER VEHICLE                      715236
SPORT UTILITY / STATION WAGON          313500
Sedan                                  153232
Station Wagon/Sport Utility Vehicle    123879
TAXI                                    50670
VAN                                     26540
OTHER                                   23982
PICK-UP TRUCK                           23069
UNKNOWN                                 19929
Taxi                                    15850
SMALL COM VEH(4 TIRES)                  14559
LARGE COM VEH(6 OR MORE TIRES)          14527
BUS                                     14057
LIVERY VEHICLE                          10481
Pick-up Truck                           10311
Box Truck                                8071
Bus                                      6640
MOTORCYCLE                               6536
BICYCLE                                  5568
Bike                                     3939
Name: vehicle_type_code1, dtype: int64

In [10]:
datanyc['vehicle_type_code_3'].value_counts().head(20)

PASSENGER VEHICLE                      63655
SPORT UTILITY / STATION WAGON          33161
Sedan                                  10379
Station Wagon/Sport Utility Vehicle     8577
UNKNOWN                                 3285
TAXI                                    3218
PICK-UP TRUCK                           2292
VAN                                     1489
OTHER                                   1108
Taxi                                     657
Pick-up Truck                            583
BICYCLE                                  533
SMALL COM VEH(4 TIRES)                   479
MOTORCYCLE                               464
LARGE COM VEH(6 OR MORE TIRES)           448
LIVERY VEHICLE                           424
BUS                                      403
Box Truck                                194
Bus                                      118
Motorcycle                               100
Name: vehicle_type_code_3, dtype: int64

We believe we have a decent understanding of our data. In the next section we will modify the column names to standardize the dataset, deal with missing values, clean duplications, and generally get our dataset to the point where we use it confidently

## Transforming the Data <a id='Transforming'></a>

### What needs attention
####  [Dropping Columns](#Drop)
* Some columns (such as vehicle_type_code_4, contributing_factor_vehicle_5) are nearly entirely empty. We'll remove those. 
* We will not be using some columns (e.g. unique_key, on_street_name, off_street_name, cross_street_name) so we can drop them completely. 


#### [Renaming Data](#Renaming)
* Cleaning and combining duplicated rows
* Renaming some columns
* Correcting misspellings
* Dealing with missing values
* Some dtype changes

#### [Redundant Columns](#Redundant)
* 'Latitude' and 'longitude' columns seem to be contained in the 'location' column. We like keeping the two values separate for now, so we can probably remove 'location' later.

#### [Data Types](#data_type)
* The values that we expect to be 'datetime' type are object ('date' and 'time' columns). We'll fix those.
* We will change the data type of zip code to string.

#### STEP 5
* We are curious to see if collisions go up seasonally, so we'll make a new variable that bins the collisions by Spring (March, April, May), Summer (June, July, August), Fall (September, October, November), and Winter (December, January, February).

#### STEP 6
* We will do some general analyses to understand more about the data set.

Let's start! Just in case we mess anything up, we'll do all our cleaning in a new DataFrame called clean_nyc.

## Dropping Columns  <a id='Drop'></a>
We'll begin by removing some columns, keeping only those missing fewer than 30% of their values. We can also drop some columns we know we're not going to use. Those operations are simple enough that we'll do them all before checking in again on the DataFrame.

In [11]:
clean_nyc = datanyc.dropna(thresh=(0.30 * datanyc.shape[0]), axis=1).copy()

In [12]:
clean_nyc.drop(columns=["collision_id", "on_street_name", "off_street_name"], inplace=True)

Let's take a peek at what clean_nyc looks like now, as far as data types and number of columns (and values in those columns):

In [13]:
clean_nyc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600038 entries, 0 to 1600037
Data columns (total 19 columns):
date                             1600038 non-null object
time                             1600038 non-null object
borough                          1115106 non-null object
zip_code                         1114912 non-null object
latitude                         1404135 non-null float64
longitude                        1404135 non-null float64
location                         1404135 non-null object
number_of_persons_injured        1600021 non-null float64
number_of_persons_killed         1600007 non-null float64
number_of_pedestrians_injured    1600038 non-null int64
number_of_pedestrians_killed     1600038 non-null int64
number_of_cyclist_injured        1600038 non-null int64
number_of_cyclist_killed         1600038 non-null int64
number_of_motorist_injured       1600038 non-null int64
number_of_motorist_killed        1600038 non-null int64
contributing_factor_vehicle_1    1

And what about the percentage of the missing values now?

In [14]:
pd.set_option('display.max_columns', 29)
clean_nyc.isnull().mean().round(4) * 100

date                              0.00
time                              0.00
borough                          30.31
zip_code                         30.32
latitude                         12.24
longitude                        12.24
location                         12.24
number_of_persons_injured         0.00
number_of_persons_killed          0.00
number_of_pedestrians_injured     0.00
number_of_pedestrians_killed      0.00
number_of_cyclist_injured         0.00
number_of_cyclist_killed          0.00
number_of_motorist_injured        0.00
number_of_motorist_killed         0.00
contributing_factor_vehicle_1     0.26
contributing_factor_vehicle_2    13.43
vehicle_type_code1                0.33
vehicle_type_code2               16.47
dtype: float64

So far, so good.

## Correcting Misspellings and Renaming  <a id='Renaming'></a>

We will modify the column names to standardize the dataset using the rename function.

In [15]:
datanyc.rename(columns={'vehicle_type_code1':'vehicle_type_code_1',
                        'vehicle_type_code2':'vehicle_type_code_2',
                       }, 
               inplace=True)

Let's take a closer look at vehicle_type_code_1.

In [16]:
datanyc['vehicle_type_code_1'].value_counts().head(40)

PASSENGER VEHICLE                      715236
SPORT UTILITY / STATION WAGON          313500
Sedan                                  153232
Station Wagon/Sport Utility Vehicle    123879
TAXI                                    50670
VAN                                     26540
OTHER                                   23982
PICK-UP TRUCK                           23069
UNKNOWN                                 19929
Taxi                                    15850
SMALL COM VEH(4 TIRES)                  14559
LARGE COM VEH(6 OR MORE TIRES)          14527
BUS                                     14057
LIVERY VEHICLE                          10481
Pick-up Truck                           10311
Box Truck                                8071
Bus                                      6640
MOTORCYCLE                               6536
BICYCLE                                  5568
Bike                                     3939
Tractor Truck Diesel                     3541
Van                               

It looks like there are misspellings and duplicates. Let's see if we can combine some of the obvious missspellings.

In [17]:
datanyc['vehicle_type_code_1'].replace('SPORT UTILITY / STATION WAGON', 'SUV', inplace=True)
datanyc['vehicle_type_code_1'].replace('Station Wagon/Sport Utility Vehicle', 'SUV', inplace=True)
datanyc['vehicle_type_code_1'].replace('TAXI', 'taxi', inplace=True)
datanyc['vehicle_type_code_1'].replace('Bike', 'BICYCLE', inplace=True)
datanyc['vehicle_type_code_1'].replace('VAN', 'Van', inplace=True)
datanyc['vehicle_type_code_1'].replace('Motorscooter', 'SCOOTER', inplace=True)
datanyc['vehicle_type_code_1'].replace('Moped', 'SCOOTER', inplace=True)
datanyc['vehicle_type_code_1'].replace('van', 'Van', inplace=True)
datanyc['vehicle_type_code_1'].replace('MOTORCYCLE', 'Motorcycle', inplace=True)
datanyc['vehicle_type_code_1'].replace('AMBULANCE', 'Ambulance', inplace=True)
datanyc['vehicle_type_code_1'].replace('Refrigerated Van', 'Van', inplace=True)
datanyc['vehicle_type_code_1'].replace('PICK-UP TRUCK', 'Pick-up Truck', inplace=True)
datanyc['vehicle_type_code_1'].replace('Motorbike', 'Motorcycle', inplace=True)
datanyc['vehicle_type_code_1'].replace('AMBUL', 'Ambulance', inplace=True)
datanyc['vehicle_type_code_1'].replace('CAB', 'taxi', inplace=True)
datanyc['vehicle_type_code_1'].replace('Cab', 'taxi', inplace=True)
datanyc['vehicle_type_code_1'].replace('VAN T', 'Van', inplace=True)
datanyc['vehicle_type_code_1'].replace('VAN/T', 'Van', inplace=True)
datanyc['vehicle_type_code_1'].replace('van t', 'Van', inplace=True)
datanyc['vehicle_type_code_1'].replace('VAN', 'Van', inplace=True)
datanyc['vehicle_type_code_1'].replace('Ambul', 'Ambulance', inplace=True)
datanyc['vehicle_type_code_1'].replace('AMB', 'Ambulance', inplace=True)
datanyc['vehicle_type_code_1'].replace('Ambu', 'Ambulance', inplace=True)
datanyc['vehicle_type_code_1'].replace('ambul', 'Ambulance', inplace=True)
datanyc['vehicle_type_code_1'].replace('Ambu', 'Ambulance', inplace=True)
datanyc['vehicle_type_code_1'].replace('Fire', 'FIRE TRUCK', inplace=True)
datanyc['vehicle_type_code_1'].replace('fire', 'FIRE TRUCK', inplace=True)
datanyc['vehicle_type_code_1'].replace('FIRE', 'FIRE TRUCK', inplace=True)
datanyc['vehicle_type_code_1'].replace('FIRET', 'FIRE TRUCK', inplace=True)
datanyc['vehicle_type_code_1'].replace('FDNY', 'FIRE TRUCK', inplace=True)
datanyc['vehicle_type_code_1'].replace('Other', 'Unknown', inplace=True)
datanyc['vehicle_type_code_1'].replace('BUS', 'Bus', inplace=True)
datanyc['vehicle_type_code_1'].replace('Box T', 'Box Truck', inplace=True)
datanyc['vehicle_type_code_1'].replace('GARBA', 'Garbage or Refuse', inplace=True)
datanyc['vehicle_type_code_1'].replace('Taxi', 'taxy', inplace=True)
datanyc['vehicle_type_code_1'].replace('taxy', 'taxi', inplace=True)
datanyc['vehicle_type_code_1'].replace('AM', 'Ambulance', inplace=True)
datanyc['vehicle_type_code_1'].replace('VN', 'Van', inplace=True)
datanyc['vehicle_type_code_1'].replace('CONV', 'Convertible', inplace=True)
datanyc['vehicle_type_code_1'].replace('Garbage or Refuse', 'Dump', inplace=True)
datanyc['vehicle_type_code_1'].replace('OTHER', 'UNKNOWN', inplace=True)
datanyc['vehicle_type_code_1'].value_counts().head(50)

PASSENGER VEHICLE                 715236
SUV                               437385
Sedan                             153232
taxi                               66521
UNKNOWN                            43911
Pick-up Truck                      33380
Van                                30321
Bus                                20697
SMALL COM VEH(4 TIRES)             14559
LARGE COM VEH(6 OR MORE TIRES)     14527
LIVERY VEHICLE                     10481
BICYCLE                             9507
Motorcycle                          8642
Box Truck                           8074
Ambulance                           3987
Tractor Truck Diesel                3541
TK                                  2485
BU                                  2229
Dump                                1941
Convertible                         1744
FIRE TRUCK                          1055
DS                                  1006
4 dr sedan                           859
PK                                   834
Flat Bed        

Now let's look at contributing_factor_vehicle_1.

In [18]:
datanyc['contributing_factor_vehicle_1'].unique()

array(['Unspecified', 'Fatigued/Drowsy', 'Driver Inattention/Distraction',
       'Other Vehicular', 'Backing Unsafely', 'Lost Consciousness',
       'Animals Action', 'Failure to Yield Right-of-Way',
       'Turning Improperly', 'Outside Car Distraction',
       'Prescription Medication', 'Oversized Vehicle',
       'Physical Disability', 'Obstruction/Debris', 'Driver Inexperience',
       'Unsafe Speed', 'Brakes Defective', 'Alcohol Involvement',
       'Traffic Control Device Improper/Non-Working',
       'Pavement Defective', 'Traffic Control Disregarded',
       'Steering Failure', 'Following Too Closely',
       'Failure to Keep Right', 'Other Electronic Device',
       'Cell Phone (hand-held)', nan, 'Glare',
       'Reaction to Other Uninvolved Vehicle',
       'Aggressive Driving/Road Rage', 'Passenger Distraction',
       'Pavement Slippery', 'Illness', 'Unsafe Lane Changing',
       'Accelerator Defective', 'Tire Failure/Inadequate', 'Fell Asleep',
       'Pedestrian/Bicyclis

There are some misspellings (e.g. "Illnes" missspelled in addition to 'Illness', in 'Drugs (illegal)' and 'Drugs (Illegal)'). Let's fix those.

In [19]:
datanyc['contributing_factor_vehicle_1'].replace('Illnes', 'Illness', inplace=True)
datanyc['contributing_factor_vehicle_1'].replace('Drugs (illegal)', 'Drugs (Illegal)', inplace=True)
datanyc['contributing_factor_vehicle_1'].unique()

array(['Unspecified', 'Fatigued/Drowsy', 'Driver Inattention/Distraction',
       'Other Vehicular', 'Backing Unsafely', 'Lost Consciousness',
       'Animals Action', 'Failure to Yield Right-of-Way',
       'Turning Improperly', 'Outside Car Distraction',
       'Prescription Medication', 'Oversized Vehicle',
       'Physical Disability', 'Obstruction/Debris', 'Driver Inexperience',
       'Unsafe Speed', 'Brakes Defective', 'Alcohol Involvement',
       'Traffic Control Device Improper/Non-Working',
       'Pavement Defective', 'Traffic Control Disregarded',
       'Steering Failure', 'Following Too Closely',
       'Failure to Keep Right', 'Other Electronic Device',
       'Cell Phone (hand-held)', nan, 'Glare',
       'Reaction to Other Uninvolved Vehicle',
       'Aggressive Driving/Road Rage', 'Passenger Distraction',
       'Pavement Slippery', 'Illness', 'Unsafe Lane Changing',
       'Accelerator Defective', 'Tire Failure/Inadequate', 'Fell Asleep',
       'Pedestrian/Bicyclis

In [20]:
datanyc['contributing_factor_vehicle_2'].unique()

array(['Unspecified', nan, 'Other Vehicular',
       'Driver Inattention/Distraction', 'Driver Inexperience',
       'Lost Consciousness', 'Turning Improperly', 'Physical Disability',
       'Failure to Yield Right-of-Way', 'Fatigued/Drowsy',
       'Passenger Distraction', 'Backing Unsafely',
       'Reaction to Other Uninvolved Vehicle', 'Prescription Medication',
       'Alcohol Involvement', 'Aggressive Driving/Road Rage', 'Illness',
       'Outside Car Distraction', 'Traffic Control Disregarded',
       'Pavement Slippery', 'Oversized Vehicle', 'Fell Asleep', 'Glare',
       'View Obstructed/Limited', 'Unsafe Speed',
       'Other Electronic Device', 'Obstruction/Debris',
       'Passing Too Closely', 'Steering Failure',
       'Lane Marking Improper/Inadequate', 'Failure to Keep Right',
       'Cell Phone (hands-free)', 'Following Too Closely',
       'Brakes Defective', 'Drugs (Illegal)', 'Tow Hitch Defective',
       'Traffic Control Device Improper/Non-Working',
       'Passin

contributing_factor_vehicle_2 had the same problem with 'Illnes' being missspelled, so we will fix it now.

In [21]:
datanyc['contributing_factor_vehicle_2'].replace('Illnes', 'Illness', inplace=True)

Even in one column you can see how much variation there is. We would suggest that whoever created this dataset turns this from a "fill in the blank" text field to a select field from a predetermined loist, to get better data fidelity, and we hope by this point, you can see why.

Next, we will rename some of the columns to make things easier while analyzing the data.

In [22]:
clean_nyc.rename(columns={'number_of_persons_injured' : 'persons_injured',
                        'number_of_persons_killed' : 'persons_killed',
                        'number_of_pedestrians_injured' : 'pedestrians_injured',
                        'number_of_pedestrians_killed' : 'pedestrians_killed',
                        'number_of_cyclist_injured' : 'cyclist_injured',
                        'number_of_cyclist_killed' : 'cyclist_killed',
                        'number_of_motorist_injured'  : 'motorist_injured',
                        'number_of_motorist_killed' : 'motorist_killed'},inplace=True)
clean_nyc.head()

,date,time,borough,zip_code,latitude,longitude,location,persons_injured,persons_killed,pedestrians_injured,pedestrians_killed,cyclist_injured,cyclist_killed,motorist_injured,motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code1,vehicle_type_code2
0,2013-10-05T00:00:00.000,17:00,QUEENS,11355,40.754769,-73.829960,POINT (-73.8299602 40.7547687),0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE
1,2013-10-27T00:00:00.000,8:00,QUEENS,11004,40.743511,-73.717405,POINT (-73.7174046 40.7435109),0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON
2,2013-10-15T00:00:00.000,10:30,MANHATTAN,10013,40.723121,-74.006234,POINT (-74.0062341 40.7231208),0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,PASSENGER VEHICLE,VAN
3,2013-10-29T00:00:00.000,15:00,BROOKLYN,11220,40.633235,-74.011907,POINT (-74.0119074 40.6332345),0.0,0.0,0,0,0,0,0,0,Fatigued/Drowsy,Unspecified,SPORT UTILITY / STATION WAGON,UNKNOWN
4,2013-10-06T00:00:00.000,5:52,NaN,NaN,40.853248,-73.835281,POINT (-73.8352807 40.8532485),1.0,0.0,0,0,0,0,1,0,Unspecified,NaN,PASSENGER VEHICLE,NaN


Great! We can see that 'persons_injured' and 'persons_killed' column values are float. Let's change them from float to integer. To do so, we will first get rid of any missing values in the 'persons_injured' and 'persons_killed' columns.

In [23]:
clean_nyc.dropna(subset = ['persons_injured'], how='all', inplace=True)
clean_nyc.dropna(subset = ['persons_killed'], how='all', inplace=True)
clean_nyc['persons_injured'] = clean_nyc.persons_injured.astype(int)
clean_nyc['persons_killed'] = clean_nyc.persons_killed.astype(int)
clean_nyc.head()

,date,time,borough,zip_code,latitude,longitude,location,persons_injured,persons_killed,pedestrians_injured,pedestrians_killed,cyclist_injured,cyclist_killed,motorist_injured,motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code1,vehicle_type_code2
0,2013-10-05T00:00:00.000,17:00,QUEENS,11355,40.754769,-73.829960,POINT (-73.8299602 40.7547687),0,0,0,0,0,0,0,0,Unspecified,Unspecified,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE
1,2013-10-27T00:00:00.000,8:00,QUEENS,11004,40.743511,-73.717405,POINT (-73.7174046 40.7435109),0,0,0,0,0,0,0,0,Unspecified,Unspecified,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON
2,2013-10-15T00:00:00.000,10:30,MANHATTAN,10013,40.723121,-74.006234,POINT (-74.0062341 40.7231208),0,0,0,0,0,0,0,0,Unspecified,Unspecified,PASSENGER VEHICLE,VAN
3,2013-10-29T00:00:00.000,15:00,BROOKLYN,11220,40.633235,-74.011907,POINT (-74.0119074 40.6332345),0,0,0,0,0,0,0,0,Fatigued/Drowsy,Unspecified,SPORT UTILITY / STATION WAGON,UNKNOWN
4,2013-10-06T00:00:00.000,5:52,NaN,NaN,40.853248,-73.835281,POINT (-73.8352807 40.8532485),1,0,0,0,0,0,1,0,Unspecified,NaN,PASSENGER VEHICLE,NaN


Next, we want to change all our values to lower case letters.

In [24]:
clean_nyc1 = clean_nyc.applymap(lambda s:s.lower() if type(s) == str else s)
clean_nyc1.head()

,date,time,borough,zip_code,latitude,longitude,location,persons_injured,persons_killed,pedestrians_injured,pedestrians_killed,cyclist_injured,cyclist_killed,motorist_injured,motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code1,vehicle_type_code2
0,2013-10-05t00:00:00.000,17:00,queens,11355,40.754769,-73.829960,point (-73.8299602 40.7547687),0,0,0,0,0,0,0,0,unspecified,unspecified,sport utility / station wagon,passenger vehicle
1,2013-10-27t00:00:00.000,8:00,queens,11004,40.743511,-73.717405,point (-73.7174046 40.7435109),0,0,0,0,0,0,0,0,unspecified,unspecified,passenger vehicle,sport utility / station wagon
2,2013-10-15t00:00:00.000,10:30,manhattan,10013,40.723121,-74.006234,point (-74.0062341 40.7231208),0,0,0,0,0,0,0,0,unspecified,unspecified,passenger vehicle,van
3,2013-10-29t00:00:00.000,15:00,brooklyn,11220,40.633235,-74.011907,point (-74.0119074 40.6332345),0,0,0,0,0,0,0,0,fatigued/drowsy,unspecified,sport utility / station wagon,unknown
4,2013-10-06t00:00:00.000,5:52,NaN,NaN,40.853248,-73.835281,point (-73.8352807 40.8532485),1,0,0,0,0,0,1,0,unspecified,NaN,passenger vehicle,NaN


Now we want to do something with the missing values. Let's see what values we have in the 'contributing_factor_vehicle_1' and 'contributing_factor_vehicle_2' columns.

In [25]:
clean_nyc1['contributing_factor_vehicle_1'].value_counts(dropna=False)

unspecified                                              588934
driver inattention/distraction                           297166
failure to yield right-of-way                             90829
following too closely                                     79352
backing unsafely                                          61181
other vehicular                                           50878
fatigued/drowsy                                           46856
turning improperly                                        40927
passing or lane usage improper                            37480
passing too closely                                       34351
unsafe lane changing                                      30447
traffic control disregarded                               24410
driver inexperience                                       23480
lost consciousness                                        19619
pavement slippery                                         15821
prescription medication                 

We can combine the missing and unspecified values.

In [26]:
clean_nyc1['contributing_factor_vehicle_1'].replace('NaN', 'unspecified', inplace=True)
clean_nyc1['contributing_factor_vehicle_1'].value_counts(dropna=False)

unspecified                                              588934
driver inattention/distraction                           297166
failure to yield right-of-way                             90829
following too closely                                     79352
backing unsafely                                          61181
other vehicular                                           50878
fatigued/drowsy                                           46856
turning improperly                                        40927
passing or lane usage improper                            37480
passing too closely                                       34351
unsafe lane changing                                      30447
traffic control disregarded                               24410
driver inexperience                                       23480
lost consciousness                                        19619
pavement slippery                                         15821
prescription medication                 

Let's see the values for the 'contributing_factor_vehicle_2' column.

In [27]:
clean_nyc1['contributing_factor_vehicle_2'].value_counts(dropna=False)

unspecified                                              1164246
NaN                                                       214838
driver inattention/distraction                             72889
other vehicular                                            26675
failure to yield right-of-way                              13922
following too closely                                      13448
fatigued/drowsy                                            10824
passing or lane usage improper                              9343
turning improperly                                          7701
backing unsafely                                            6989
passing too closely                                         6542
driver inexperience                                         5451
traffic control disregarded                                 5285
unsafe lane changing                                        5237
lost consciousness                                          5223
pavement slippery        

We can do the same for this column as well.

In [28]:
clean_nyc1['contributing_factor_vehicle_1'].replace('NaN', 'unspecified', inplace=True)
clean_nyc1['contributing_factor_vehicle_1'].value_counts(dropna=False)

unspecified                                              588934
driver inattention/distraction                           297166
failure to yield right-of-way                             90829
following too closely                                     79352
backing unsafely                                          61181
other vehicular                                           50878
fatigued/drowsy                                           46856
turning improperly                                        40927
passing or lane usage improper                            37480
passing too closely                                       34351
unsafe lane changing                                      30447
traffic control disregarded                               24410
driver inexperience                                       23480
lost consciousness                                        19619
pavement slippery                                         15821
prescription medication                 

## Redundant Coumns: 'latitude' and 'longitude' <a id='Redundant'></a>

We suspect the 'location' column is simply a concatenation of 'latitude' and 'longitude' columns.

In [29]:
clean_nyc[["latitude", "longitude", "location"]].head(30)

,latitude,longitude,location
0,40.754769,-73.829960,POINT (-73.8299602 40.7547687)
1,40.743511,-73.717405,POINT (-73.7174046 40.7435109)
2,40.723121,-74.006234,POINT (-74.0062341 40.7231208)
3,40.633235,-74.011907,POINT (-74.0119074 40.6332345)
4,40.853248,-73.835281,POINT (-73.8352807 40.8532485)
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,40.817899,-73.938090,POINT (-73.9380903 40.8178994)
8,40.705628,-73.915512,POINT (-73.9155117 40.7056278)
9,40.704403,-73.814439,POINT (-73.8144393 40.7044031)


Before we quickly do something we might regret, let's first see if it's true that all the location data follows the same pattern we see right now:

In [30]:
clean_nyc['location'].str.match('POINT \(-7\d\.\d+ \d{2}\.\d+\)', na=False).value_counts()

True     1402946
False     197056
Name: location, dtype: int64

1402517 rows where that's not the case! More than we expected! Let's check them out!

In [31]:
clean_nyc[~clean_nyc['location'].str.match('POINT \(-7\d\.\d+ \d{2}\.\d+\)', na=False)].head(30)

,date,time,borough,zip_code,latitude,longitude,location,persons_injured,persons_killed,pedestrians_injured,pedestrians_killed,cyclist_injured,cyclist_killed,motorist_injured,motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code1,vehicle_type_code2
5,2013-10-29T00:00:00.000,22:00,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,1,0,Unspecified,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE
6,2013-10-30T00:00:00.000,14:30,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,UNKNOWN,VAN
16,2013-10-26T00:00:00.000,0:25,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,Fatigued/Drowsy,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE
26,2013-10-29T00:00:00.000,9:20,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,Unspecified,Unspecified,OTHER,OTHER
31,2013-10-13T00:00:00.000,22:16,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,Unspecified,Unspecified,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON
32,2013-10-15T00:00:00.000,8:30,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,Unspecified,Unspecified,SPORT UTILITY / STATION WAGON,SPORT UTILITY / STATION WAGON
40,2013-10-03T00:00:00.000,7:45,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,Unspecified,Unspecified,PASSENGER VEHICLE,UNKNOWN
41,2013-10-28T00:00:00.000,8:05,NaN,NaN,NaN,NaN,NaN,1,0,1,0,0,0,0,0,Driver Inattention/Distraction,NaN,PASSENGER VEHICLE,NaN
44,2013-10-23T00:00:00.000,13:20,NaN,NaN,NaN,NaN,NaN,2,0,0,0,0,0,2,0,Failure to Yield Right-of-Way,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE
49,2013-10-29T00:00:00.000,19:05,NaN,NaN,NaN,NaN,NaN,1,0,0,0,1,0,0,0,Unspecified,Unspecified,OTHER,BICYCLE


Ahh those sweet missing values... As we've seen before, latitude, longitude	and location columns have 12% of their values missing each. While we're at it, let's make some changes with the help of `fillna` function.

In [32]:
clean_nyc['location'] = clean_nyc['location'].fillna(0)
clean_nyc['latitude'] = clean_nyc['latitude'].fillna(0)
clean_nyc['longitude'] = clean_nyc['longitude'].fillna(0)

In [33]:
clean_nyc[~clean_nyc['location'].str.match('POINT \(-7\d\.\d+ \d{2}\.\d+\)', na=False)].head(30)

,date,time,borough,zip_code,latitude,longitude,location,persons_injured,persons_killed,pedestrians_injured,pedestrians_killed,cyclist_injured,cyclist_killed,motorist_injured,motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code1,vehicle_type_code2
5,2013-10-29T00:00:00.000,22:00,NaN,NaN,0.0,0.0,0,1,0,0,0,0,0,1,0,Unspecified,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE
6,2013-10-30T00:00:00.000,14:30,NaN,NaN,0.0,0.0,0,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,UNKNOWN,VAN
16,2013-10-26T00:00:00.000,0:25,NaN,NaN,0.0,0.0,0,0,0,0,0,0,0,0,0,Fatigued/Drowsy,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE
26,2013-10-29T00:00:00.000,9:20,NaN,NaN,0.0,0.0,0,0,0,0,0,0,0,0,0,Unspecified,Unspecified,OTHER,OTHER
31,2013-10-13T00:00:00.000,22:16,NaN,NaN,0.0,0.0,0,0,0,0,0,0,0,0,0,Unspecified,Unspecified,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON
32,2013-10-15T00:00:00.000,8:30,NaN,NaN,0.0,0.0,0,0,0,0,0,0,0,0,0,Unspecified,Unspecified,SPORT UTILITY / STATION WAGON,SPORT UTILITY / STATION WAGON
40,2013-10-03T00:00:00.000,7:45,NaN,NaN,0.0,0.0,0,0,0,0,0,0,0,0,0,Unspecified,Unspecified,PASSENGER VEHICLE,UNKNOWN
41,2013-10-28T00:00:00.000,8:05,NaN,NaN,0.0,0.0,0,1,0,1,0,0,0,0,0,Driver Inattention/Distraction,NaN,PASSENGER VEHICLE,NaN
44,2013-10-23T00:00:00.000,13:20,NaN,NaN,0.0,0.0,0,2,0,0,0,0,0,2,0,Failure to Yield Right-of-Way,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE
49,2013-10-29T00:00:00.000,19:05,NaN,NaN,0.0,0.0,0,1,0,0,0,1,0,0,0,Unspecified,Unspecified,OTHER,BICYCLE


Now all NaN's in the latitude and longitude columns are replaced with 0, let's change latitude, let's drop the location column.

In [34]:
clean_nyc.drop(columns="location", inplace = True)

And let's peek at our data.

In [35]:
clean_nyc.head()

,date,time,borough,zip_code,latitude,longitude,persons_injured,persons_killed,pedestrians_injured,pedestrians_killed,cyclist_injured,cyclist_killed,motorist_injured,motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code1,vehicle_type_code2
0,2013-10-05T00:00:00.000,17:00,QUEENS,11355,40.754769,-73.829960,0,0,0,0,0,0,0,0,Unspecified,Unspecified,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE
1,2013-10-27T00:00:00.000,8:00,QUEENS,11004,40.743511,-73.717405,0,0,0,0,0,0,0,0,Unspecified,Unspecified,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON
2,2013-10-15T00:00:00.000,10:30,MANHATTAN,10013,40.723121,-74.006234,0,0,0,0,0,0,0,0,Unspecified,Unspecified,PASSENGER VEHICLE,VAN
3,2013-10-29T00:00:00.000,15:00,BROOKLYN,11220,40.633235,-74.011907,0,0,0,0,0,0,0,0,Fatigued/Drowsy,Unspecified,SPORT UTILITY / STATION WAGON,UNKNOWN
4,2013-10-06T00:00:00.000,5:52,NaN,NaN,40.853248,-73.835281,1,0,0,0,0,0,1,0,Unspecified,NaN,PASSENGER VEHICLE,NaN


In [36]:
clean_nyc['latitude'] = clean_nyc['latitude'].fillna(0)
clean_nyc['longitude'] = clean_nyc['longitude'].fillna(0)
clean_nyc.head()

,date,time,borough,zip_code,latitude,longitude,persons_injured,persons_killed,pedestrians_injured,pedestrians_killed,cyclist_injured,cyclist_killed,motorist_injured,motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code1,vehicle_type_code2
0,2013-10-05T00:00:00.000,17:00,QUEENS,11355,40.754769,-73.829960,0,0,0,0,0,0,0,0,Unspecified,Unspecified,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE
1,2013-10-27T00:00:00.000,8:00,QUEENS,11004,40.743511,-73.717405,0,0,0,0,0,0,0,0,Unspecified,Unspecified,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON
2,2013-10-15T00:00:00.000,10:30,MANHATTAN,10013,40.723121,-74.006234,0,0,0,0,0,0,0,0,Unspecified,Unspecified,PASSENGER VEHICLE,VAN
3,2013-10-29T00:00:00.000,15:00,BROOKLYN,11220,40.633235,-74.011907,0,0,0,0,0,0,0,0,Fatigued/Drowsy,Unspecified,SPORT UTILITY / STATION WAGON,UNKNOWN
4,2013-10-06T00:00:00.000,5:52,NaN,NaN,40.853248,-73.835281,1,0,0,0,0,0,1,0,Unspecified,NaN,PASSENGER VEHICLE,NaN


## Data Type <a id='data_type'></a>

Let's look at our dates to make sure they are all in the same format:

In [37]:
clean_nyc[['date', 'time']].head()

,date,time
0,2013-10-05T00:00:00.000,17:00
1,2013-10-27T00:00:00.000,8:00
2,2013-10-15T00:00:00.000,10:30
3,2013-10-29T00:00:00.000,15:00
4,2013-10-06T00:00:00.000,5:52


The 'date' column definitely needs some fixing... We will transform the string timestamp for 'date' to a true datetime data type.

In [38]:
clean_nyc['date'] = pd.to_datetime(datanyc['date'])
clean_nyc.head()

,date,time,borough,zip_code,latitude,longitude,persons_injured,persons_killed,pedestrians_injured,pedestrians_killed,cyclist_injured,cyclist_killed,motorist_injured,motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code1,vehicle_type_code2
0,2013-10-05,17:00,QUEENS,11355,40.754769,-73.829960,0,0,0,0,0,0,0,0,Unspecified,Unspecified,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE
1,2013-10-27,8:00,QUEENS,11004,40.743511,-73.717405,0,0,0,0,0,0,0,0,Unspecified,Unspecified,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON
2,2013-10-15,10:30,MANHATTAN,10013,40.723121,-74.006234,0,0,0,0,0,0,0,0,Unspecified,Unspecified,PASSENGER VEHICLE,VAN
3,2013-10-29,15:00,BROOKLYN,11220,40.633235,-74.011907,0,0,0,0,0,0,0,0,Fatigued/Drowsy,Unspecified,SPORT UTILITY / STATION WAGON,UNKNOWN
4,2013-10-06,5:52,NaN,NaN,40.853248,-73.835281,1,0,0,0,0,0,1,0,Unspecified,NaN,PASSENGER VEHICLE,NaN


We also want to create a new column which will carry the values for hours only. We think that can be helpful when grouping the times and visualizing the data. We will create a new column called 'hour' in which we will only have the hours instead of hours and minutes.

In [ ]:
clean_nyc['time'] = pd.to_datetime(clean_nyc.time)
clean_nyc['hour'] = clean_nyc['time'].dt.hour
clean_nyc.head(30)

We'll also clean up the Zip Codes here, which should be a string, not an integer

In [ ]:
clean_nyc.loc[:,'zip_code'] = clean_nyc['zip_code'].astype(str)

## Step 5: Making a Seasons Variable

We are interested in adding a variable that gives the season in which something occurred.

In [ ]:
clean_nyc1['date'] = pd.to_datetime(datanyc.date)
clean_nyc1['date'].dt.month.head(30)

In [ ]:
def season(date):
    if date.month in ([3, 4, 5]):
        val = 'Spring'
    elif date.month in ([6, 7, 8]):
        val = 'Summer'
    elif date.month in ([9, 10, 11]):
        val = 'Autumn'
    elif date.month in ([12, 1, 2]):
        val = 'Winter'
    else:
        val = "Unspecified"
    return val

clean_nyc1['season'] = clean_nyc1['date'].apply(season)

In [ ]:
clean_nyc1['season'].value_counts()

## Step 6: General Analysis

Let's see, for example, how many persons, cyclist, pedestrian and motorist killed in 3000000 collisions.

In [ ]:
personskillednum = clean_nyc1['persons_killed'] > 0
personskillednum.sum()

In [ ]:
cyclistkillednum = clean_nyc1['cyclist_killed'] > 0
cyclistkillednum.sum()

In [ ]:
motoristkillednum = clean_nyc1['motorist_killed'] > 0
motoristkillednum.sum()

In [ ]:
pedestrianskillednum = clean_nyc1['pedestrians_killed'] > 0
pedestrianskillednum.sum()

It looks like being a pedestrian is not very safe in NYC. What about injured people? 

In [ ]:
personsinjurednum = clean_nyc1['persons_injured'] > 0
personsinjurednum.sum()

In [ ]:
cyclistinjurednum = clean_nyc1['cyclist_injured'] > 0
cyclistinjurednum.sum()

In [ ]:
motoristinjurednum = clean_nyc1['motorist_injured'] > 0
motoristinjurednum.sum()

In [ ]:
pedestriansinjurednum = clean_nyc1['pedestrians_injured'] > 0
pedestriansinjurednum.sum()

Interestingly, as far as injuries go, being a motorist is worse than being a pedestrian. So maybe we can say that there are many motorcycle collisions that result in injuries but not death?  

Let's see which boroughs have a high number of collisions.

In [ ]:
datanyc['borough'].value_counts()

A lot of collisions in Brooklyn! 

Let's see the common reasons behind collisions.

In [ ]:
datanyc.contributing_factor_vehicle_1.value_counts().head(30)

Driver inattention/distraction and following too closely are the main causes of collisions. Do we need more proof why we shouldn't "send a quick text" while driving!?

What about the vehicle types? We will first make a copy of data -- a whole new data frame named contributing_factor to see the common vehicle types involved in driver inattention/distraction caused collisions.

In [ ]:
contributing_factor = datanyc[datanyc['contributing_factor_vehicle_1'] == "Driver Inattention/Distraction"].copy()
contributing_factor.head()

In [ ]:
contributing_factor['vehicle_type_code_1'].value_counts().head(30)

Passenger vehicles take the lead in terms of vehicles most commonly involced in driver inattention/distraction caused collisions. They are followed by sport utilities/station wagons. 

## Cleaning Conclusion

We have a more precise set of data to work with in clean_nyc. Let's take a peek at it.

In [ ]:
clean_nyc1.shape

In [ ]:
datanyc.shape

We've reduced our data by 9 columns and around 36 rows. Let's take a general look at our data as well as the 'info'.

In [ ]:
clean_nyc1.head()

In [ ]:
clean_nyc1.info()

We've done a lot of data cleaning, and this is a great start for our next stage. We'll save our data locally.

In [ ]:
clean_nyc1.to_csv("clean_nyc1_data.csv", index = False)

Now, we can begin our visualizations...

## Data Visualizations

We will create:
* An exploratory vizualization of fatal crash frequency over time
* An exploratory vizualization of contributing factors to crash fatalities
* An exploratory vizualization of fatalities to pedestrians vs cyclists vs motorist
* An exploratory vizualization of average of deaths by borough
* An exploratory vizualization of average number of deaths by hour
* An exploratory vizualization of pedestrian injuries in Brooklyn due to driver inexperience

## Fatal crash frequency over time

We'll look at a few of the contributing factor attributes to get a sense of the data using the unique function.

In [ ]:
clean_nyc1['contributing_factor_vehicle_1'].unique()

This hgave us a good insight into the recorded contributing factor values, but we'd like to get a better sense of how populated each of the 5 columns actually look, or if there are trends we should look at.

Next we'll use the value_counts function to see how frequently these values appear in the data.

In [ ]:
clean_nyc1['contributing_factor_vehicle_1'].value_counts()

In [ ]:
clean_nyc1['contributing_factor_vehicle_2'].value_counts()

For now, we'll just analyze the first contributing factor, to get a general sense.

We need to refine our dataframe. We'll try to sort rows where fatalities occured into a new dataframe, then we need to group by date and sum.

In [ ]:
lethal_crashes = clean_nyc1[clean_nyc1['persons_killed'] > 0]
lethal_crashes.head()

Now to group by Date. We are going to use a number of chained functions here. We are adding up the number of fatalities per day, and sorting cronologically.

In [ ]:
clean_nyc1.groupby('date').persons_killed.sum().reset_index().head(20)

Next, we'll port this into a new dataframe with some other fields we are interested in for analysis, then try to use matplotlib.

In [ ]:
counted_lethal_crashes = clean_nyc1.groupby(['date','borough', 'contributing_factor_vehicle_1']).persons_killed.sum().reset_index()
counted_lethal_crashes.info()

Let's visualize it!

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(20,10)) # Create the plot space upon which to plot the data
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling

x = counted_lethal_crashes['date']
y = counted_lethal_crashes['persons_killed']

ax.plot(x,y,
        linestyle='',
        marker='x',
        alpha = 0.5,
        color='Red'
       )

# Set title and labels for axes
ax.set(xlabel="Date",
       ylabel="Killed Count",
       title="Number of people killed by motorists in NYC")

plt.show()

We have succesfully plotted the Car crash fatalities in NYC over time!

We can see from this that the majority of crashes have either 0 or 1 persons killed. Though there are some outliers. We could plug other columns into this chart, and see distributions of pedestrians, cyclists, motorists killed, but it will be easier and more visually appealing with pandas and seaborn. Here it is hard to get a precise view of the data, though it does help us know where to look.

## Contributing factors to crash fatalities

Next we'll look into the causes of fatal car crashes. We'll start by creating a new dataframe that groups those causes.

In [ ]:
counted_lethal_crash_factors = clean_nyc1.groupby('contributing_factor_vehicle_1').persons_killed.sum().reset_index().sort_values("persons_killed", ascending = False)
counted_lethal_crash_factors.head()

Finally, let's drop 0's if we have any.

In [ ]:
counted_lethal_crash_factors.tail()

Yes, let's remove those zero values now.

In [ ]:
counted_lethal_crash_factors = counted_lethal_crash_factors[counted_lethal_crash_factors.persons_killed != 0]
counted_lethal_crash_factors.tail()

Great. Now let's Visualize Fatal Car Crash Causes in NYC.

In [ ]:
counted_lethal_crash_factors.plot.barh("contributing_factor_vehicle_1", 
                                      "persons_killed",
                                      figsize = (10,20)
                                      ).invert_yaxis()

This horizontal bar chart gives a slightly better sense of the data, that fatal accidents are mostly caused by driver's bad behaviours, or are unrecorded.

This is of course only looking at fatal accidents, only at the first recorded cause, and the reports are likely not 100% reliable. However, if NYC's government wanted to do more to reduce fatalities, this is useful information.

## Fatalities to pedestrians vs cyclists vs motorist

We'll switch to Seaborn for the rest of our work, which allows for greater range and easier user controls.

For this we'll compare the distributions of fatalities by the category of person killed. We'll group each category from our original dataframe into it's own dataframe. We will then add a column to each dataframe which creates a running total of persons killed, using the cumsum function.

In [ ]:
killed_motorists = clean_nyc1.groupby('date').motorist_killed.sum().reset_index()
killed_motorists['sum'] = killed_motorists['motorist_killed'].cumsum()

killed_cyclist = clean_nyc1.groupby('date').cyclist_killed.sum().reset_index()
killed_cyclist['sum'] = killed_cyclist['cyclist_killed'].cumsum()

killed_pedestrians = clean_nyc1.groupby('date').pedestrians_killed.sum().reset_index()
killed_pedestrians['sum'] = killed_pedestrians['pedestrians_killed'].cumsum()

Now we can plot these using Seaborn, a library built to handle python data visualizations.

For this graph, we will overlay the rolling sum of fatalities (grouped by travel type) on the same graph, to get a sense of relative danger.

In [ ]:
import seaborn as sns
sns.set(style="whitegrid")

fig, ax = plt.subplots(figsize=(20, 10))

sns.lineplot(x='date', y='sum', color = 'red', data=killed_motorists, ax=ax)
sns.lineplot( x='date', y='sum', color = 'blue', data=killed_cyclist, ax=ax)
sns.lineplot( x='date', y='sum', color = 'green', data=killed_pedestrians, ax=ax)
ax.legend(['Motorists', 'Cyclists', 'Pedestrians'], facecolor='w')

plt.show()

This is interesting. More pedestrians are killed than motorists by car crashes. I guess that is not entirely surprising, but it is disheartening.

This analysis lends itself well to comparing injury rates as well, which we would expect to be significantly higher.

Assuming there was more time, further analysis into the rates of change in the fatality set (to see if NYC is getting safer or more dangerous) would be manageable with another function over these data sets.

## Average of deaths by borough

Let's see the average of killed persons by borough with the collisions count.

In [ ]:
clean_nyc1.groupby('borough').persons_killed.agg(['count', 'mean'])

Let's graph this with a bar chart.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
clean_nyc1.groupby('borough').persons_killed.mean().plot(kind='bar', figsize=(10, 6))
plt.ylabel('average persons killed')
plt.title('Average of Deaths by Borough', fontsize=15)
plt.xticks(rotation=0, fontsize='large')
plt.xlabel(' ')

It looks like Staten Island and Queens have the highest number of deaths, meanwhile Manhattan has the lowest number of deaths, which can be due to its smaller size.

## Average number of deaths by hour

Let's see the mean of persons killed by hour. We are interested in seeing the percentage rate of persons killed for each hour. We want to see this information hourly to get a better general idea. To do so, we will first drop the missing values in the 'date' and 'time' columns and change their data types from integer to string. Then, create a new column named 'combined', which will combine 'date' and 'time' columns into the same panda series. Finally, take that series and convert it to datetime data type and store it in a new column called 'combined_date_time'. That will allow us to get the hours when graphing it.

In [ ]:
clean_nyc1.dropna(subset = ['date'], how='all', inplace=True)
clean_nyc1.dropna(subset = ['time'], how='all', inplace=True)
clean_nyc1['date'] = clean_nyc1.date.astype(str)
clean_nyc1['time'] = clean_nyc1.time.astype(str)
clean_nyc1.head()

In [ ]:
combined = clean_nyc1.date.str.cat(clean_nyc1.time, sep= ' ')
combined.head()

In [ ]:
clean_nyc1['combined_date_time'] = pd.to_datetime(combined)

In [ ]:
clean_nyc1.dtypes

In [ ]:
clean_nyc1.groupby(clean_nyc1.combined_date_time.dt.hour).persons_killed.mean().plot(figsize=(10, 6))
plt.title('Average Number of Deaths by Hour', fontsize=18)
plt.ylabel('average number of persons killed', fontsize=13)
plt.xlabel('time', fontsize=13)

It looks like number of deaths goes up around 4:00 in the morning and 23:00 in the evening. It decreases during the day between 8:00 and 14:00, which are typical work hours.

## Pedestrian injuries in Brooklyn due to driver inexperience

Finally, we will import the data set from nypd motor vehicle crash and limit 30,000 rows and looking at the values containing Brooklyn (borough of occurance) and Driver Inexperience (for contributing factor vehicle 1) This will allow us to work with some of the dataset and look at crashes involving driver inexperience in the borough of Brooklyn.

In [ ]:
mv_data = pd.read_csv('https://data.cityofnewyork.us/resource/h9gi-nx95.csv?$limit=30000&borough=BROOKLYN&contributing_factor_vehicle_1=Driver%20Inexperience')


Let's check it.

In [ ]:
mv_data.head()

Now we can find the unique types of vehicles involved in these crashes. Let's look at vehcile number one, the first vehicle impacting. This column value is vehicle_type_code1.

In [ ]:
mv_data['vehicle_type_code1'].unique()

We can further narrow what we are looking at by selecting for vehicle types. We can look at Station Wagon/Sport Utility Vehicle and Sedan. So, in Brooklyn for inexperienced drivers, what can we see for this group driving Station Wagon/Sport Utility Vehicle versus driving a sedan.

In [ ]:
mv_data['vehicle_type_code1'].value_counts().head(25)

So for Brooklyn and crashes due to inexperienced drivers, passenger vehicle, sports utility/station wagon, sedan, and then station wagon/sport utility were the largest number of vehicles involved in the crashes. We could maybe combine sports utility/station wagon and station wagon/sport utility groups.

In [ ]:
mv_data['vehicle_type_code1'] = mv_data['vehicle_type_code1'].str.replace('SPORT UTILITY / STATION WAGON', 'SUV')
mv_data['vehicle_type_code1'] = mv_data['vehicle_type_code1'].str.replace('Station Wagon/Sport Utility Vehicle', 'SUV')

Let's look at this same information with seaborn data visualization tool.

In [ ]:
f, ax = plt.subplots(figsize=(15, 25))
sns.scatterplot(x="contributing_factor_vehicle_1", y="vehicle_type_code1", 
               hue="number_of_pedestrians_injured", linewidth=0, data=mv_data, ax=ax)

So, it looks like for this particular item, crashes with driver inexperience, most vehicles did not have pedestrian injuries. Livery vehicles had 1 pedestrian injury and "motor" (what is motor compared to motor vehicle, I'm not sure) had 1 pedestrian injury.

# <center> <br>[Begining of the page](#Top)</center> <a id='Bottom'></a>